In [ ]:
import json
from nltk.tag import StanfordNERTagger
from nltk import word_tokenize
import os
import sys

# Change these paths
java_path = "/Library/Java/JavaVirtualMachines/jdk-10.0.1.jdk/Contents/Home/bin/java.exec"
jar = '/Users/gretchenstahlman/stanford-ner-2018-02-27/stanford-ner.jar'
model = '/Users/gretchenstahlman/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz'
directory = "/Users/gretchenstahlman/Documents/Geoparsing/Expanding Access"

os.environ['JAVAHOME'] = java_path
ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

out_dict = {}

t = 0
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read() 
            text = ner_tagger.tag(word_tokenize(text))
            locations = []
            current_loc = []
            is_prev_loc = 0
            for word in text:
                if word[1] == "LOCATION":
                    if is_prev_loc == 1:
                        current_loc.append(word[0])
                    else:
                        locations.append(current_loc)
                        current_loc = [word[0]]
                        is_prev_loc = 1
                else:
                    is_prev_loc = 0
            loc_list = []
            for i in locations[1:]:
                if len(i) > 1:
                    exp = " ".join(locations[1])
                    loc_list.append(exp)
                else:
                    loc_list.append(i)
        out_dict[filename] = loc_list
        f.close()
        t += 1
        print(str(t) + " files have been processed.")

with open(directory + '/geo.json', 'w') as outfile:
    json.dump(out_dict, outfile)

# Chunk to detect unreadable files
import os
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r') as f:
            try: text = f.read() 
            except: 
                print(filename + "error")
                continue